In [5]:
import io
import requests
from pymongo import MongoClient, UpdateOne
from pprint import pprint
import re
import json
from time import sleep

In [3]:
uri = open('./../../mongo_uri.txt', 'r', encoding='utf-8').read()
client = MongoClient(uri)
print(client)
db = client.boletines_db
# pprint(db.boletines.find_one())

MongoClient(host=['clustershido-shard-00-02-uxkjr.gcp.mongodb.net:27017', 'clustershido-shard-00-01-uxkjr.gcp.mongodb.net:27017', 'clustershido-shard-00-00-uxkjr.gcp.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', authsource='admin', replicaset='ClusterShido-shard-0', ssl=True)


In [27]:
cursor = db.boletines.find(
    {}, 
    {'boletin':1, 'encabezado':1, 'sumarios':1}
).sort('fecha')

# Agrego punto final a oraciones que no tienen
patron = re.compile(r'([^\.])(\n|$)')
remplazo = r'\1.\n'

In [28]:
def peticion_freeling(archivo):
    files = { 'file': archivo }
    return requests.post(
        "http://www.corpus.unam.mx/servicio-freeling/analyze.php", 
        files = files, 
        params = { 
            'outf': 'tagged', 
            'format': 'json'
        }
    ).json() # hace la petición y manda el archivo en formato json

In [29]:
%%time
updates = []
i=0

for boletin in cursor:
    filtro = {'boletin_id': boletin['_id']}
    update = {'$set' : {}}
    
    # Encabezado
    archivo = io.StringIO(patron.sub(remplazo, boletin['encabezado'])) 
    # Enviar petición
    sentences_enc = peticion_freeling(archivo)
    update['$set']['encabezado'] = sentences_enc
    # Duermo para no darle demasiadas requests por segundo al server
    sleep(1)
    
    # Sumarios
    try:
        archivo = io.StringIO(
            patron.sub(remplazo, '\n'.join(boletin['sumarios']))
        ) 
        sentences_sum = peticion_freeling(archivo)
        update['$set']['sumarios'] = sentences_sum
        sleep(1)
    except KeyError:
        sentences_sum = []
    
    # Encabezado
    archivo = io.StringIO(boletin['boletin']) 
    # Enviar petición
    sentences_bol = peticion_freeling(archivo)
    update['$set']['boletin'] = sentences_bol
    # Duermo para no darle demasiadas requests por segundo al server
    sleep(1)
    
    if i%35==0:
    #Ejemplo, obtener todos los lemas
        for sentences in [sentences_enc, sentences_sum, sentences_bol]:
            for i_sent, sentence in enumerate(sentences):
                print(f'Oración {i_sent+1}')
                tokens = [w['token'] for w in sentence]
                print('  ', ' '.join(tokens))
            print('-----------')
        print('°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°')
    
    updates.append(UpdateOne(filtro, update, upsert = True)) 
    i+=1
cursor.rewind()

Oración 1
   José_Antonio_Meade presentó la Coordinación_de_Atención_Ciudadana y Buzón_de_el_Candidato .
-----------
Oración 1
   El abanderado presidencial refrendó su compromiso de encabezar un gobierno a la medida de cada persona .
-----------
Oración 1
   El candidato de la coalición Todos_por_México a la Presidencia_de_la_República , José_Antonio_Meade , presentó la Coordinación_de_Atención_Ciudadana y el Buzón_de_el_Candidato .
Oración 2
   A_través_de estos mecanismos , Meade y su esposa , Juana_Cuevas , recabarán las propuestas , opiniones , inquietudes y necesidades de la población .
Oración 3
   El equipo de Atención_Ciudadana estará conformado por jóvenes voluntarios , quienes apoyan el proyecto de gobierno de el candidato de la coalición Todos_por_México y desean colaborar en la construcción de un país más justo y equitativo , donde se cierren las brechas y se erradiquen las desigualdades .
Oración 4
   El voluntariado escuchará a las personas , comprenderá sus problemática

Oración 1
   José_Antonio_Meade se comprometió a construir un mejor destino para Coahuila .
-----------
-----------
Oración 1
   El candidato de la coalición Todos_por_México , José_Antonio_Meade , se comprometió a forjar un mejor destino para Coahuila , a el desarrollo de la infraestructura en el estado y a hacer equipo con las mujeres y los jóvenes para ganar la elección de el próximo primero de julio .
Oración 2
   En un evento masivo en Saltillo , ante más de 10_mil simpatizantes que gritaban “ Pepe presidente , Pepe presidente ” , el aspirante de el PRI , PVEM y Nueva_Alianza dijo que en Coahuila son invencibles y anunció que , como presidente , será el mejor aliado de el gobernador Miguel_Riquelme para avanzar en el desarrollo de la entidad .
Oración 3
   José_Antonio_Meade se comprometió con la ampliación a seis carriles de la autopista Saltillo-Monterrey ; así_como a invertir en la modernización de la que une Piedras_Negras con Laredo .
Oración 4
   Acompañado por su esposa Jua

Oración 1
   Observa AMLO que Anaya le fue peor que a Meade y “ nosotros para arriba como la espuma de el mar ” .
-----------
Oración 1
   Invita a Alicia_Bárcena para que cuando concluya su función en la CAPAL sea la embajadora en la Organización_de_las_Naciones_Unidas .
Oración 2
   Agradece a la cantante Belinda su apoyo y su solidaridad , es una extraordinaria artista .
Oración 3
   Convoca AMLO a dirigentes de el MORENA , PT y PES para enseñar a ciudadanos cómo votar el próximo_1_de_julio .
-----------
Oración 1
   Andrés_Manuel_López_Obrador observó que en el segundo debate le fue peor a Anaya que a Meade , porque Anaya quedó a el descubierto como un mentecato , un mentiroso y “ nosotros para arriba como la espuma de el mar ” .
Oración 2
   En entrevista y a la pregunta de los reporteros que ayer se apagó el amor y paz , porque respondió a los señalamientos de Anaya y Meade , el candidato por la coalición “ Juntos haremos historia ” manifestó que se pasó de la raya el candidato d

Oración 1
   Convoca AMLO a reforzar el trabajo de defensa de el voto y tener representantes en todas las casillas .
-----------
Oración 1
   Pide no confiar se , porque se enfrente a una mafia de el poder , es gente que no tiene escrúpulos morales , hacen fraudes electorales .
-----------
Oración 1
   El candidato por la coalición “ Juntos haremos historia ” por la Presidencia_de_México , Andrés_Manuel_López_Obrador convocó a quienes trabajan en la defensa de los votos a reforzar el trabajo de organización y tener representantes en todas las casillas .
Oración 2
   Pidió no confiar se , porque se enfrente a una mafia de el poder , es gente que no tiene escrúpulos morales , hacen fraudes electorales , no se respeta la voluntad de los ciudadanos , y ya saben que cómo compran votos , rellenan las urnas , falsifican las actas , y todo esto se les facilita si no se tiene organización .
Oración 3
   A un mes de la elección presidencial , llamó a cubrir todas las casillas y reforzar de maner

In [30]:
res = db.stats_freeling.bulk_write(updates)
pprint(res.bulk_api_result)

{'nInserted': 0,
 'nMatched': 2,
 'nModified': 0,
 'nRemoved': 0,
 'nUpserted': 345,
 'upserted': [{'_id': ObjectId('5ec937df07772823d5599887'), 'index': 0},
              {'_id': ObjectId('5ec937df07772823d5599888'), 'index': 1},
              {'_id': ObjectId('5ec937df07772823d5599889'), 'index': 2},
              {'_id': ObjectId('5ec937df07772823d559988a'), 'index': 3},
              {'_id': ObjectId('5ec937df07772823d559988b'), 'index': 4},
              {'_id': ObjectId('5ec937df07772823d559988c'), 'index': 5},
              {'_id': ObjectId('5ec937df07772823d559988d'), 'index': 6},
              {'_id': ObjectId('5ec937df07772823d559988e'), 'index': 7},
              {'_id': ObjectId('5ec937df07772823d559988f'), 'index': 8},
              {'_id': ObjectId('5ec937df07772823d5599890'), 'index': 9},
              {'_id': ObjectId('5ec937df07772823d5599891'), 'index': 11},
              {'_id': ObjectId('5ec937df07772823d5599892'), 'index': 12},
              {'_id': ObjectId('5ec93

In [31]:
client.close()

In [5]:
# test = "En un pronunciamiento especial, el candidato de la coalición Todos por México condenó enérgicamente los abusos —suscitados a consecuencia de la llamada política de “Tolerancia Cero”—, que incluyen la separación de niños y padres, así como el confinamiento de menores en jaulas. Meade llamó a la Organización de las Naciones Unidas y a Unicef a intervenir para detener los atropellos."
# files = {'file': io.StringIO(test)}
# #Parámetros
# params = {'outf': 'tagged', 'format': 'json'}
# #Enviar petición
# url = "http://www.corpus.unam.mx/servicio-freeling/analyze.php"
# r = requests.post(url, files=files, params=params)
# #Convertir de formato json
# obj = r.json()
    
# with open('./out.json', 'w', encoding='utf-8') as wf:
#     json.dump(obj, wf)